> # Hazard modeling for passing plays

The yardage gained on the passing play in American football distributes to one-dimenstional distribution, which depends on defensive and offensive formations. This notebook adopts **Cox proportional hazard model** to express this distribution and shows the advantage of hazard modeling approach to infer players' contributions in each play.

# Introduction

Hazard modeling mainly focus on explaining the duration of time until some events happen. In hazard modeling, we estimate the hazard function instead of probability density function. 

Let $y$ be obtained yardage. Since the obtained yardage takes a discrete value, our aim is to estimate the discrete hazard function

$$
\begin{aligned}
h(y) = \mathrm{Pr}(y \leq Y < y + 1 \mid Y \geq y),
\end{aligned}
$$

which explains the probability of a player with a ball being stopped at $y$.

In a passing play, defensive players try to intercept pass or stop the player who catch the ball as soon as possible. In terms of the hazard function, defensive players try to increase $h(r)$ whereas offensive players decrease $h(r)$. 

The aim of this notebook is to estimate how each player affects on this hazard. It is not sufficient to evaluate players contribution of passing play by simple stats like obtained yards, touchdowns and tuckles. Even one wide reciever gains long yards, it is unclear how other offensive players contribute to this gain.
On the other hand, using hazard values enables us to compare players' contributions between diferrent positions. 

# Model definition

We adopt **Cox proportional model** to express the hazard function. Cox proportional model divides this hazard function into two parts as

$$
\begin{aligned}
    h(y) = h_0(y) \cdot \exp(\phi(x, y)), \, \text{$x$ : covariates.}
\end{aligned}
$$


The former part indicates how the hazard function depends on time and the latter part indicates how it depends on the covariate information.  Estimating the former part in empirical manner enables us to express the complex distribution easily.

We define $\phi(x, y)$ as the summation of player's contribution.
$$
\begin{aligned}
    \phi(x, y) =  \phi_\mathrm{P}(x_\mathrm{P}, y \mid x) + \sum_{i=1}^{10} \phi_\mathrm{O}(x^{(i)}_\mathrm{O}, y \mid x) + \sum_{j=1}^{11} \phi_\mathrm{D}(x^{(j)}_\mathrm{D}, y \mid x).
\end{aligned}
$$
Here, $x_\mathrm{P}, \{x^{(i)}_\mathrm{O}\}_{i=1, \cdots, 10} \,\, ,  \{x^{(j)}_\mathrm{D}\}_{j=1, \cdots, 11} \, \, $ are player's information for a quaterback, offensive players and defensive players. $\phi_\mathrm{P}(\cdot), \phi_\mathrm{O}(\cdot), \phi_\mathrm{D}(\cdot)$ are estimand functions. Specifically, we use players' locations and velocities when the quaterback throws the ball as $x$.

Each player's contribution to the hazard is also affected by nearby players. We express these interactions as graph expression. 
First, we calculate Delaunay diagram from players location those who directly involve passing play (e.g. QB, WL, LB, DB). Then, we omit some edges which may not influence on this passing play, and construct a graph indicates the pairs of offensive players and defensive players.
Under this graph, we adopt **Gated Graph Neural Network** for expressing $\phi_\mathrm{P}(\cdot), \phi_\mathrm{O}(\cdot), \phi_\mathrm{D}(\cdot)$. 

In [ ]:
from IPython.display import Image
Image("../input/gatedgnn/GGNN.png")

# Application example

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tf
import networkx as nx

import matplotlib.pyplot as plt
import matplotlib.patches as patches
from scipy.spatial import Delaunay

In [ ]:
def split_dispaly_name(display_name):
    display_name_splited = display_name.split(' ')
    if len(display_name_splited) == 2:
        return display_name_splited[0][0] + '.' + display_name_splited[1]
    else:
        return display_name


def find_passer_and_carrier(game, play, play_detail):

    homeTeam, awayTeam = game.homeTeamAbbr.values[0], game.visitorTeamAbbr.values[0]
    possessionTeam = play.possessionTeam.values[0]
    play_description = play.playDescription.values[0].replace('(', '').replace(').', '').replace(' .', '')

    player_in_description =  [word for word in play_description.split(' ') if '.' in word]
    player_in_description_converted = []

    for player in player_in_description:

        player_splited = player.split('.')
        player_in_description_converted.append(player_splited[0][0] + '.' + player_splited[1])

    display_names = np.array(list(map(split_dispaly_name, play_detail.displayName.values)))
    play_detail_in_descritpion = play_detail[np.array(list(map(lambda display_name: display_name in player_in_description_converted, display_names)))]

    play_detail_passer = play_detail_in_descritpion[play_detail_in_descritpion.position.values == 'QB']

    if homeTeam == possessionTeam:
        play_detail_carrier = play_detail_in_descritpion[(play_detail_in_descritpion.position.values != 'QB') * (play_detail_in_descritpion.team.values == 'home')]
    else:
        play_detail_carrier = play_detail_in_descritpion[(play_detail_in_descritpion.position.values != 'QB') * (play_detail_in_descritpion.team.values == 'away')]

    return play_detail_passer, play_detail_carrier


def extract_feature(game, play, play_detail):

    homeTeam, awayTeam = game.homeTeamAbbr.values[0], game.visitorTeamAbbr.values[0]
    possessionTeam = play.possessionTeam.values[0]

    vel = pd.concat([np.cos((- play_detail.dir.values + 90) / 360 * 2 * np.pi) * play_detail['s'], np.sin((- play_detail.dir.values + 90) / 360 * 2 * np.pi) * play_detail['s']], axis=1)
    vel.columns = ['vel_x', 'vel_y']
    play_detail = pd.concat([play_detail, vel], axis=1)

    play_detail_passer, play_detail_carrier = find_passer_and_carrier(game, play, play_detail)

    if homeTeam == possessionTeam:
        play_detail_offense, play_detail_defense = play_detail[play_detail.team.values == "home"], play_detail[play_detail.team.values == "away"]
    else:
        play_detail_offense, play_detail_defense = play_detail[play_detail.team.values == "away"], play_detail[play_detail.team.values == "home"]

    direction = 2 * np.float('right' == play_detail.playDirection.values[0]) - 1
    scrimmageLine = play.absoluteYardlineNumber.values[0]
    if np.isnan(scrimmageLine):
        scrimmageLine = play_detail['x'].values.mean(0)

    name_passer = play_detail_passer.displayName.values
    name_carrier = play_detail_carrier.displayName.values
    name_offense = play_detail_offense.displayName.values
    name_defense = play_detail_defense.displayName.values

    is_not_passer_carrier = np.logical_not(np.any(name_offense == name_passer[:, np.newaxis], 0)) * np.logical_not(np.any(name_offense == name_carrier[:, np.newaxis], 0))

    name = np.hstack([name_passer, name_carrier, np.pad(name_offense[is_not_passer_carrier], [0, 11 - name_offense.shape[0]]), np.pad(name_defense, [0, 11 - name_defense.shape[0]])])
    name_replaced = np.array(['Other' if na == 0 else na for na in name])

    position_passer = play_detail_passer.position.values
    position_carrier = play_detail_carrier.position.values
    position_offense = play_detail_offense.position.values
    position_defense = play_detail_defense.position.values

    position = np.hstack([position_passer, position_carrier, np.pad(position_offense[is_not_passer_carrier], [0, 11 - position_offense.shape[0]]), np.pad(position_defense, [0, 11 - position_defense.shape[0]])])
    position_replaced = []

    for pos in position:

        if pos in ['QB']:
            position_replaced.append('QB')
        elif pos in  ['RB', 'HB', 'FB', 'TE', 'WR']:
            position_replaced.append('WR')
        elif pos in ['LB', 'ILB', 'OLB', 'MLB']:
            position_replaced.append('LB')
        elif pos in ['DB', 'CB', 'FS', 'SS', 'S', 'SAF']:
            position_replaced.append('DB')
        else:
            position_replaced.append('Other')

    position_replaced = np.array(position_replaced)

    loc_passer = (play_detail_passer[['x', 'y']].values - np.array([scrimmageLine, 53.3 / 2])) * direction
    loc_carrier = (play_detail_carrier[['x', 'y']].values - np.array([scrimmageLine, 53.3 / 2])) * direction
    loc_offense = (play_detail_offense[['x', 'y']].values - np.array([scrimmageLine, 53.3 / 2])) * direction
    loc_defense = (play_detail_defense[['x', 'y']].values - np.array([scrimmageLine, 53.3 / 2])) * direction

    vel_passer = play_detail_passer[['vel_x', 'vel_y']].values * direction
    vel_carrier = play_detail_carrier[['vel_x', 'vel_y']].values * direction
    vel_offense = play_detail_offense[['vel_x', 'vel_y']].values * direction
    vel_defense = play_detail_defense[['vel_x', 'vel_y']].values * direction

    if possessionTeam == homeTeam:
        offense_team, defense_team = homeTeam, awayTeam
    else:
        defense_team, offense_team = homeTeam, awayTeam

    loc = np.vstack([loc_passer, loc_carrier, np.pad(loc_offense[is_not_passer_carrier], ([0, 11 - loc_offense.shape[0]], [0, 0])), np.pad(loc_defense, ([0, 11 - loc_defense.shape[0]], [0, 0]))])
    vel = np.vstack([vel_passer, vel_carrier, np.pad(vel_offense[is_not_passer_carrier], ([0, 11 - loc_offense.shape[0]], [0, 0])), np.pad(vel_defense, ([0, 11 - loc_defense.shape[0]], [0, 0]))])
    m = np.hstack([np.pad(np.ones(loc_offense.shape[0]), ([0, 11 - loc_offense.shape[0]])), np.pad(np.ones(loc_defense.shape[0]), ([0, 11 - loc_defense.shape[0]]))]).astype(np.bool)

    x = np.hstack([loc, vel])

    delau = Delaunay(loc[m])
    adj = np.zeros((22, 22))
    ind = np.arange(22)[m]

    for i in range(delau.simplices.shape[0]):
        for j in delau.simplices[i]:
            adj[ind[j], ind[delau.simplices[i]]] = 1

    adj *= 1 - np.eye(22)
    adj[1:11, 1:11] = 0.
    adj[11:, 11:] = 0.

    y = play.offensePlayResult.values[0]
    c = 'touchdown' not in play.playDescription.values[0]

    return x, m, adj, y, c, position_replaced, name_replaced, offense_team, defense_team


In [ ]:
games = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/games.csv')
plays = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/plays.csv')
players = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/players.csv')

weeks = []
for i in range(1, 18):
    week = pd.read_csv('/kaggle/input/nfl-big-data-bowl-2021/week%s.csv' %i)
    week = week.query('event == "pass_forward" or event == "pass_shovel" or event == "pass_lateral"')
    week = week.drop_duplicates()
    weeks.append(week)

xs, ms, adjs, ys, cs = [], [], [], [], []

positions, names = [], []
offense_teams, defense_teams = [], []

for week in weeks:

    for gameId in list(set(week.gameId.values)):

        for playId in list(set(plays.query('gameId == %s' % gameId).playId.values)):

            game = games.query('gameId == %s' % gameId)
            play = plays.query('gameId == %s and playId == %s' % (gameId, playId))
            play_detail = week.query('gameId == %s and playId == %s' % (gameId, playId))

            if (play.passResult.values[0] == 'C' or play.passResult.values[0] == 'I') and play.penaltyCodes.values[0] is np.nan and 'FUMBLES' not in play.playDescription.values[0] and 'Direct snap' not in play.playDescription.values[0] and 'Punt' not in play.playDescription.values[0] and 'spike' not in play.playDescription.values[0]:

                try:
                    x, m, adj, y, c, position, name, offense_team, defense_team = extract_feature(game, play, play_detail)

                    xs.append(x)
                    ms.append(m)
                    adjs.append(adj)
                    ys.append(y)
                    cs.append(c)

                    positions.append(position)
                    names.append(name)
                    offense_teams.append(offense_team)
                    defense_teams.append(defense_team)

                except:
                    
                    pass

xs, adjs, ys, cs = np.stack(xs), np.stack(adjs), np.hstack(ys), np.array(cs).astype(np.int)
ms = np.stack(ms)

names, positions, offense_teams, defense_teams = np.vstack(names),  np.vstack(positions), np.hstack(offense_teams), np.hstack(defense_teams)

ind = np.logical_not(np.any(np.isnan(xs), axis=(1, 2))) * (ms[:, :11].sum(1) != 0) * (ms[:, 11:].sum(1) != 0)
xs, adjs, ys, cs, ms = xs[ind], adjs[ind], ys[ind], cs[ind], ms[ind]
names, positions, offense_teams, defense_teams = names[ind], positions[ind], offense_teams[ind], defense_teams[ind]

X_eval, A_eval, M_eval = tf.constant(xs, dtype=tf.float32), tf.constant(adjs, dtype=tf.float32), tf.constant(ms, dtype=tf.float32)

xs, adjs, ys, cs, ms = np.vstack([xs, xs * np.array([1, -1, 1, -1])]), np.vstack([adjs, adjs]), np.hstack([ys, ys]), np.hstack([cs, cs]), np.vstack([ms, ms])
xs, adjs, ys, cs, ms = xs[np.argsort(ys)], adjs[np.argsort(ys)], ys[np.argsort(ys)], cs[np.argsort(ys)], ms[np.argsort(ys)]

X, A, M = tf.constant(xs, dtype=tf.float32), tf.constant(adjs, dtype=tf.float32), tf.constant(ms, dtype=tf.float32)

n = xs.shape[0]
ys_unique, ys_index, ys_inverse, ys_count = np.unique(ys, return_index=True, return_inverse=True, return_counts=True)
ys_mask = tf.constant(np.arange(ys_unique[0], ys_unique[-1] + 1)[:, np.newaxis] == ys_unique, dtype=tf.float32)

cs_count = []
for j in ys_unique:
    cs_count.append(cs[ys == j].sum())

cs_count = np.array(cs_count)

cs_mask = np.zeros((n, ys_unique.shape[0]))
for j, index, count in zip(range(ys_unique.shape[0]), ys_index, ys_count):
    cs_mask[index:index+count, j] = 1.
cs_mask = tf.constant(cs_mask, dtype=tf.float32)

mask = tf.constant(ys_index <= np.arange(n)[:, np.newaxis], dtype=tf.float32)
inf_array = - tf.ones_like(mask, dtype=tf.float32) * np.inf

zero_index = tf.cast(ys_unique == 0, tf.float32)[tf.newaxis]

In [ ]:
class GGNN(tf.keras.Model):

    
    def __init__(self):

        super(GGNN, self).__init__()

        self.gru_L = 5

        self.denseGRU_P, self.denseGRU_O, self.denseGRU_D = tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.tanh), tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.tanh), tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.tanh)

        self.update_P, self.update_O, self.update_D = tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.sigmoid, use_bias=False), tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.sigmoid, use_bias=False), tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.sigmoid, use_bias=False)
        self.reset_P, self.reset_O, self.reset_D = tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.sigmoid, use_bias=False), tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.sigmoid, use_bias=False), tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.sigmoid, use_bias=False)
        self.modify_P, self.modify_O, self.modify_D = tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.tanh, use_bias=True), tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.tanh, use_bias=True), tf.keras.layers.Dense(n_layerGRU, activation=tf.nn.tanh, use_bias=True)

        self.dropoutGRU_P, self.dropoutGRU_O, self.dropoutGRU_D = tf.keras.layers.Dropout(dropout_rate), tf.keras.layers.Dropout(dropout_rate), tf.keras.layers.Dropout(dropout_rate)
        self.dropoutGRU_neighbor_P, self.dropoutGRU_neighbor_O, self.dropoutGRU_neighbor_D = tf.keras.layers.Dropout(dropout_rate), tf.keras.layers.Dropout(dropout_rate), tf.keras.layers.Dropout(dropout_rate)

        self.dense1_P, self.dense1_O, self.dense1_D = tf.keras.layers.Dense(n_layer1, activation=tf.nn.tanh), tf.keras.layers.Dense(n_layer1, activation=tf.nn.tanh), tf.keras.layers.Dense(n_layer1, activation=tf.nn.tanh)
        self.dropout1_P, self.dropout1_O, self.dropout1_D = tf.keras.layers.Dropout(dropout_rate), tf.keras.layers.Dropout(dropout_rate), tf.keras.layers.Dropout(dropout_rate)

        self.dense2_P, self.dense2_O, self.dense2_D = tf.keras.layers.Dense(1, use_bias=False), tf.keras.layers.Dense(1, use_bias=False), tf.keras.layers.Dense(1, use_bias=False)
        self.weight_P, self.weight_O, self.weight_D = tf.keras.layers.Dense(1, activation=tf.nn.sigmoid), tf.keras.layers.Dense(1, activation=tf.nn.sigmoid), tf.keras.layers.Dense(1, activation=tf.nn.sigmoid)


    @tf.function
    def call(self, X, A, M, is_training=False):

        X_P = tf.slice(X, [0, 0, 0], [-1, 1, -1])
        X_O = tf.concat([tf.tile(X_P, (1, 10, 1)), tf.slice(X, [0, 1, 0], [-1, 10, -1])], axis=2)
        X_D = tf.concat([tf.tile(X_P, (1, 11, 1)), tf.slice(X, [0, 11, 0], [-1, 11, -1])], axis=2)

        M_P, M_O, M_D = tf.slice(M, [0, 0], [-1, 1]), tf.slice(M, [0, 1], [-1, 10]), tf.slice(M, [0, 11], [-1, 11])

        layerGRU_P, layerGRU_O, layerGRU_D = self.denseGRU_P(X_P), self.denseGRU_O(X_O), self.denseGRU_D(X_D)

        maskGRU_update_P, maskGRU_update_O, maskGRU_update_D = self.dropoutGRU_P(tf.ones_like(layerGRU_P), is_training), self.dropoutGRU_O(tf.ones_like(layerGRU_O), is_training), self.dropoutGRU_D(tf.ones_like(layerGRU_D), is_training)
        maskGRU_reset_P, maskGRU_reset_O, maskGRU_reset_D = self.dropoutGRU_P(tf.ones_like(layerGRU_P), is_training), self.dropoutGRU_O(tf.ones_like(layerGRU_O), is_training), self.dropoutGRU_D(tf.ones_like(layerGRU_D), is_training)
        maskGRU_modify_P, maskGRU_modify_O, maskGRU_modify_D = self.dropoutGRU_P(tf.ones_like(layerGRU_P), is_training), self.dropoutGRU_O(tf.ones_like(layerGRU_O), is_training), self.dropoutGRU_D(tf.ones_like(layerGRU_D), is_training)

        maskGRU_update_neighbor_P, maskGRU_update_neighbor_O, maskGRU_update_neighbor_D = self.dropoutGRU_neighbor_P(tf.ones(tf.shape(layerGRU_P) * (1, 1, 2)), is_training), self.dropoutGRU_neighbor_O(tf.ones(tf.shape(layerGRU_O) * (1, 1, 2)), is_training), self.dropoutGRU_neighbor_D(tf.ones(tf.shape(layerGRU_D) * (1, 1, 2)), is_training)
        maskGRU_reset_neighbor_P, maskGRU_reset_neighbor_O, maskGRU_reset_neighbor_D = self.dropoutGRU_neighbor_P(tf.ones(tf.shape(layerGRU_P) * (1, 1, 2)), is_training), self.dropoutGRU_neighbor_O(tf.ones(tf.shape(layerGRU_O) * (1, 1, 2)), is_training), self.dropoutGRU_neighbor_D(tf.ones(tf.shape(layerGRU_D) * (1, 1, 2)), is_training)
        maskGRU_modify_neighbor_P, maskGRU_modify_neighbor_O, maskGRU_modify_neighbor_D = self.dropoutGRU_neighbor_P(tf.ones(tf.shape(layerGRU_P) * (1, 1, 2)), is_training), self.dropoutGRU_neighbor_O(tf.ones(tf.shape(layerGRU_O) * (1, 1, 2)), is_training), self.dropoutGRU_neighbor_D(tf.ones(tf.shape(layerGRU_D) * (1, 1, 2)), is_training)

        for l in range(self.gru_L):

            layerGRU_neighbor_P = tf.concat([tf.matmul(A[:, :1, 1:11], layerGRU_O), tf.matmul(A[:, :1, 11:], layerGRU_D)], axis=-1)
            layerGRU_neighbor_O = tf.concat([tf.matmul(A[:, 1:11, :1], layerGRU_P), tf.matmul(A[:, 1:11, 11:], layerGRU_D)], axis=-1)
            layerGRU_neighbor_D = tf.concat([tf.matmul(A[:, 11:, :1], layerGRU_P), tf.matmul(A[:, 11:, 1:11], layerGRU_O)], axis=-1)

            z_P = self.update_P(tf.concat([layerGRU_P * maskGRU_update_P, layerGRU_neighbor_P * maskGRU_update_neighbor_P], 2))
            r_P = self.reset_P(tf.concat([layerGRU_P * maskGRU_reset_P, layerGRU_neighbor_P * maskGRU_reset_neighbor_P], 2))
            layerGRU_modified_P = self.modify_P(tf.concat([layerGRU_P * r_P * maskGRU_modify_P, layerGRU_neighbor_P * maskGRU_modify_neighbor_P], 2))

            z_O = self.update_O(tf.concat([layerGRU_O * maskGRU_update_O, layerGRU_neighbor_O * maskGRU_update_neighbor_O], 2))
            r_O = self.reset_O(tf.concat([layerGRU_O * maskGRU_reset_O, layerGRU_neighbor_O * maskGRU_reset_neighbor_O], 2))
            layerGRU_modified_O = self.modify_O(tf.concat([layerGRU_O * r_O * maskGRU_modify_O, layerGRU_neighbor_O * maskGRU_modify_neighbor_O], 2))

            z_D = self.update_D(tf.concat([layerGRU_D * maskGRU_update_D, layerGRU_neighbor_D * maskGRU_update_neighbor_D], 2))
            r_D = self.reset_D(tf.concat([layerGRU_D * maskGRU_reset_D, layerGRU_neighbor_D * maskGRU_reset_neighbor_D], 2))
            layerGRU_modified_D = self.modify_D(tf.concat([layerGRU_D * r_D * maskGRU_modify_D, layerGRU_neighbor_D * maskGRU_modify_neighbor_D], 2))

            layerGRU_P = (1. - z_P) * layerGRU_P + z_P * layerGRU_modified_P
            layerGRU_O = (1. - z_O) * layerGRU_O + z_O * layerGRU_modified_O
            layerGRU_D = (1. - z_D) * layerGRU_D + z_D * layerGRU_modified_D

        layer1_P, layer1_O, layer1_D = self.dense1_P(layerGRU_P), self.dense1_O(layerGRU_O), self.dense1_D(layerGRU_D)
        layer1_P, layer1_O, layer1_D = self.dropout1_P(layer1_P), self.dropout1_O(layer1_O), self.dropout1_D(layer1_D)
        layer2_P, layer2_O, layer2_D = self.dense2_P(layer1_P), self.dense2_O(layer1_O), self.dense2_D(layer1_D)
        weight_P, weight_O, weight_D = self.weight_P(layer1_P), self.weight_O(layer1_O), self.weight_D(layer1_D)

        layer2_P = layer2_P * (weight_P * zero_index + (1. - weight_P))
        layer2_O = layer2_O * (weight_O * zero_index + (1. - weight_O))
        layer2_D = layer2_D * (weight_D * zero_index + (1. - weight_D))

        out = tf.squeeze(M_P[:, :, tf.newaxis] * layer2_P, 1) + tf.reduce_sum(M_O[:, :, tf.newaxis] * layer2_O, 1) + tf.reduce_sum(M_D[:, :, tf.newaxis] * layer2_D, 1)

        return out

    def call_players(self, X, A, M):

        X_P = tf.slice(X, [0, 0, 0], [-1, 1, -1])
        X_O = tf.concat([tf.tile(X_P, (1, 10, 1)), tf.slice(X, [0, 1, 0], [-1, 10, -1])], axis=2)
        X_D = tf.concat([tf.tile(X_P, (1, 11, 1)), tf.slice(X, [0, 11, 0], [-1, 11, -1])], axis=2)

        M_P, M_O, M_D = tf.slice(M, [0, 0], [-1, 1]), tf.slice(M, [0, 1], [-1, 10]), tf.slice(M, [0, 11], [-1, 11])

        layerGRU_P, layerGRU_O, layerGRU_D = self.denseGRU_P(X_P), self.denseGRU_O(X_O), self.denseGRU_D(X_D)

        for l in range(self.gru_L):

            layerGRU_neighbor_P = tf.concat([tf.matmul(A[:, :1, 1:11], layerGRU_O), tf.matmul(A[:, :1, 11:], layerGRU_D)], axis=-1)
            layerGRU_neighbor_O = tf.concat([tf.matmul(A[:, 1:11, :1], layerGRU_P), tf.matmul(A[:, 1:11, 11:], layerGRU_D)], axis=-1)
            layerGRU_neighbor_D = tf.concat([tf.matmul(A[:, 11:, :1], layerGRU_P), tf.matmul(A[:, 11:, 1:11], layerGRU_O)], axis=-1)

            z_P = self.update_P(tf.concat([layerGRU_P, layerGRU_neighbor_P], 2))
            r_P = self.reset_P(tf.concat([layerGRU_P, layerGRU_neighbor_P], 2))
            layerGRU_modified_P = self.modify_P(tf.concat([layerGRU_P * r_P, layerGRU_neighbor_P], 2))

            z_O = self.update_O(tf.concat([layerGRU_O, layerGRU_neighbor_O], 2))
            r_O = self.reset_O(tf.concat([layerGRU_O, layerGRU_neighbor_O], 2))
            layerGRU_modified_O = self.modify_O(tf.concat([layerGRU_O * r_O, layerGRU_neighbor_O], 2))

            z_D = self.update_D(tf.concat([layerGRU_D, layerGRU_neighbor_D], 2))
            r_D = self.reset_D(tf.concat([layerGRU_D, layerGRU_neighbor_D], 2))
            layerGRU_modified_D = self.modify_D(tf.concat([layerGRU_D * r_D, layerGRU_neighbor_D], 2))

            layerGRU_P = (1. - z_P) * layerGRU_P + z_P * layerGRU_modified_P
            layerGRU_O = (1. - z_O) * layerGRU_O + z_O * layerGRU_modified_O
            layerGRU_D = (1. - z_D) * layerGRU_D + z_D * layerGRU_modified_D

        layer1_P, layer1_O, layer1_D = self.dense1_P(layerGRU_P), self.dense1_O(layerGRU_O), self.dense1_D(layerGRU_D)
        layer2_P, layer2_O, layer2_D = self.dense2_P(layer1_P), self.dense2_O(layer1_O), self.dense2_D(layer1_D)
        weight_P, weight_O, weight_D = self.weight_P(layer1_P), self.weight_O(layer1_O), self.weight_D(layer1_D)

        layer2_P = layer2_P * (weight_P * zero_index + (1. - weight_P))
        layer2_O = layer2_O * (weight_O * zero_index + (1. - weight_O))
        layer2_D = layer2_D * (weight_D * zero_index + (1. - weight_D))

        return tf.concat([M_P[:, :, tf.newaxis] * layer2_P, M_O[:, :, tf.newaxis] * layer2_O, M_D[:, :, tf.newaxis] * layer2_D], axis=1)

    
def compute_cost(model, X, A, M):

    out = model.call(X, A, M, True)

    out_max = tf.reduce_max(tf.where(tf.cast(mask, tf.bool), out, inf_array), 0)
    exp_sum = tf.reduce_sum(tf.exp(out - out_max) * mask, 0)
    den = (out_max + tf.math.log(exp_sum)) * cs_count

    cost = - tf.reduce_sum(tf.reduce_sum(out * cs_mask, 1) * cs) + tf.reduce_sum(den)

    return cost


@tf.function
def compute_gradients(model, X, A, M):

    with tf.GradientTape() as tape:
        cost = compute_cost(model, X, A, M)

    return tape.gradient(cost, model.trainable_variables), cost


@tf.function
def apply_gradients(optimizer, gradients, variables):

    optimizer.apply_gradients(zip(gradients, variables))


def compute_baseline_hazard(model, X, A, M):

    out = model.call(X, A, M, False)

    out_max = tf.reduce_max(tf.where(tf.cast(mask, tf.bool), out, inf_array), 0)
    exp_sum = tf.reduce_sum(tf.exp(out - out_max) * mask, 0)
    den = (out_max + tf.math.log(exp_sum)) * cs_count

    baseline_hazard = np.sum(ys_mask * tf.exp(- out_max) * exp_sum.numpy() ** -1 * cs_count, axis=1)

    return baseline_hazard


def compute_hazard_ratio(model, X, A, M):

    out = model.call(X, A, M, False)
    hazard_ratio = np.dot(tf.exp(out).numpy(), tf.transpose(ys_mask))

    return hazard_ratio

In [ ]:
print('Training ...')

learning_rate = 0.01
dropout_rate = 0.1
n_layerGRU = 32
n_layer1 = 16
n_player_info = 4
n_ties = ys_unique.shape[0]

model = GGNN()
optimizer = tf.keras.optimizers.Adam(learning_rate)

traning_epochs = 1000

for epoch in range(traning_epochs):

    gradients, cost_epoch = compute_gradients(model, X, A, M)
    apply_gradients(optimizer, gradients, model.variables)

    if (epoch + 1) % 100 == 0:

        print('epoch ' + str(epoch + 1) + ': ' + str(cost_epoch.numpy()))
        
out_players = model.call_players(X_eval, A_eval, M_eval)

In [ ]:
def draw_play(game, play, play_detail, is_pass=True):

    x, m, adj, y, c, position, name, offense_team, defense_team = extract_feature(game, play, play_detail)
    out = model.call_players(tf.constant(x[np.newaxis], dtype=tf.float32), tf.constant(adj[np.newaxis], dtype=tf.float32), tf.constant(m[np.newaxis], dtype=tf.float32))

    loc, vel = x[m, :2], x[m, 2:]
    position = position[m]

    if is_pass:
        score = np.sum(out.numpy()[0, m] * zero_index, -1)
    else:
        score = np.sum(out.numpy()[0, m] * (1. - zero_index), -1) / np.sum(1. - zero_index)

    n_offense, n_defense = m[:11].sum(), m[11:].sum()
    n = n_offense + n_defense

    G = nx.Graph()

    G.add_nodes_from(np.arange(n_offense), bipartite=0)
    G.add_nodes_from(np.arange(n_offense, n_offense + n_defense), bipartite=1)
    node_color = ['r']
    node_color.extend(['b' for i in range(n_offense - 1)])
    node_color.extend(['g' for i in range(n_defense)])

    row, col = np.where(adj[m, :][:, m] != 0)
    G.add_edges_from(zip(row, col))

    plt.figure(figsize=(12, 12))

    nx.draw_networkx_nodes(G, loc, node_color=node_color, node_size=1000, alpha=1.)
    nx.draw_networkx_edges(G, loc, alpha=0.5, style='dashed', edge_color='k')
    nx.draw_networkx_labels(G, loc, {i: position[i] for i in range(len(position))}, font_weight='bold', font_color='white')

    for i in range(n):
        plt.arrow(loc[i, 0], loc[i, 1], vel[i, 0] / 2. + 0.01, vel[i, 1] / 2. + 0.01, width=0.01, head_width=0.3,head_length=0.3,length_includes_head=True, color='k', alpha=0.4)
        plt.text(loc[i, 0]+0.5, loc[i, 1]+0.5, np.around(np.exp(score[i]), 2))

    xmin, ymin = loc.min(0)
    xmax, ymax = loc.max(0)
    
    plt.vlines(0, ymin-1, ymax+1, linestyle='solid', alpha=0.2)
    plt.vlines(play.yardsToGo.values[0], ymin-1, ymax+1, color='goldenrod', linestyle='solid', alpha=0.5)
    plt.ylim(xmin-1, xmax+1)
    plt.ylim(ymin-1, ymax+1)
    
    print("OFFENSE: " + str(np.around(np.exp(np.sum(score[:11])), 2)))
    print("DEFENSE: " + str(np.around(np.exp(np.sum(score[11:])), 2)))

The following figure shows the assigned hazard values for players in one passing play. 
Each value indicates hazard ratio at 0 yard. 
For example, if a defensive player have 1.5, then the hazard at 0 yard on this play is 1.5 times larger due to this player.

In [ ]:
i = 39
gameId, playId = week.gameId.values[i], week.playId.values[i]

game = games.query('gameId == %s' % gameId)
play = plays.query('gameId == %s and playId == %s' % (gameId, playId))
play_detail = week.query('gameId == %s and playId == %s' % (gameId, playId))

draw_play(game, play, play_detail, False)

Following tables show that the mean hazard values for players and teams in 2018 season. These results differ from the passing stats in 2018 season. This is because our model only considers offensive and defensive formations and ignores the players' performances and skills.

In [ ]:
offense_players = np.unique(names[positions == 'WR'])

score_offense_players = pd.Series([np.exp(tf.reduce_mean(tf.reduce_sum(out_players[:, 1:11][names[:, 1:11] == player] * zero_index, 1) / tf.reduce_sum(zero_index))) for player in offense_players if np.sum(names == player) > 100], index= [player for player in offense_players if np.sum(names == player) > 100])
score_offense_players = score_offense_players.sort_values()

defense_players = np.unique(names[positions == 'LB'])

score_defense_players = pd.Series([np.exp(tf.reduce_mean(tf.reduce_sum(out_players[:, 11:][names[:, 11:] == player] * zero_index, 1) / tf.reduce_sum(zero_index))) for player in defense_players if np.sum(names == player) > 100], index= [player for player in defense_players if np.sum(names == player) > 100])
score_defense_players = score_defense_players.sort_values(ascending=False)

In [ ]:
print('Top 10 Offense players (WR)')
print(score_offense_players.head(10))

In [ ]:
print('Top 10 Defense players (DB)')
print(score_defense_players.head(10))

In [ ]:
teams = np.unique(offense_teams)

score_offense_teams = pd.Series([np.exp(tf.reduce_mean(tf.reduce_sum(out_players[offense_teams == team][:, :11] * (1 - zero_index), (1, 2))) / np.sum(1 - zero_index)) for team in teams], index=teams)
score_offense_teams = score_offense_teams.sort_values()

score_defense_teams = pd.Series([np.exp(tf.reduce_mean(tf.reduce_sum(out_players[defense_teams == team][:, 11:] * (1 - zero_index), (1, 2))) / np.sum(1 - zero_index)) for team in teams], index=teams)
score_defense_teams = score_defense_teams.sort_values(ascending=False)

In [ ]:
print('Top 10 Offense teams')
print(score_offense_teams.head(10))

In [ ]:
print('Top 10 Defense teams')
print(score_defense_teams.head(10))

# Conclusion

In this notebook, we have demonstrated how to incorpolate hazard modeling to evaluate passing play performance. 
The main advantage of our model is that it can decompose defensive (or offensive) performance into the sum of players' contributions in terms of hazard rate.

Our model can be extended to more complicated model. One possible challenge is to incorporate each player's skill into our model. Such extension may provide more reliable information about player's performance along one season and suggest appropriate defensive (or offensive) formations for each NFL team.